Adapted from https://github.com/akamaster/pytorch_resnet_cifar10

# ResNet Framework (resnet.py)

In [1]:
# Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

In [2]:
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

In [3]:
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [6]:
# Other ResNet-S Available: [20,32,44,*56*,110,1202]
def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])

In [7]:
resnet_dict = {
    "resnet56": resnet56
}

# Trainer/Evaluation (trainer.py)

In [8]:
# Libraries
import argparse
import os
import shutil
import time
import numpy as np
#from collections import Counter

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms.v2 as transforms
import torchvision.datasets as datasets
#import resnet # Refers to resnet.py, aka above

from sklearn.metrics import precision_recall_fscore_support as pr_fscore_mtrc

# Eliminate nondeterministic algorithm procedures
cudnn.deterministic = True

### "Parse_args" Function

In [9]:
def parse_args(pt_path="",pretrain_flag=True,finetune_flag=False,eval_flag=False,
               batch_size=128,print_freq=5,sam_flag=False,seg_path="",mask_pad_param=0):
    parser = argparse.ArgumentParser(description='Proper ResNets for CIFAR10 in pytorch')
    #model_names = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']
    model_names = ['resnet56']
    
    parser.add_argument('--arch', '-a', metavar='ARCH', default='resnet56',
                        choices=model_names,
                        help='model architecture: ' + ' | '.join(model_names) +
                        ' (default: resnet56)')
    parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                        help='number of data loading workers (default: 4)')
    # Modified to be number of epochs during ***FINETUNING***
    # Previous, from scratch training, total epochs = 200
    parser.add_argument('--epochs', default=25, type=int, metavar='N',
                        help='number of epochs to run for funetuning')
    # FOR TRAINING (from scratch)
    # parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
    #                     help='manual epoch number (useful on restarts)')
    parser.add_argument('-b', '--batch-size', default=batch_size, type=int,
                        metavar='N', help='mini-batch size (default: 128)')
    # Scheduler adjusts currently, even during finetuning, 
    # assuming 'last_epoch' parameter is used
    parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                        metavar='LR', help='initial learning rate')
    parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                        help='momentum')
    parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                        metavar='W', help='weight decay (default: 1e-4)')
    # Changed from default = 50, when training from scratch/200 epochs
    parser.add_argument('--print-freq', '-p', default=print_freq, type=int,
                        metavar='N', help='print frequency (default: 5)')
    # MANUALLY SET TO LOCATION OF RESNET56 CHECKPOINT FOR PRETRAINED MODEL
    parser.add_argument('--resume', default=pt_path, type=str, metavar='PATH',
                        help='path to latest checkpoint (default: none)')
    # FLAG
    parser.add_argument('--pt', '--pretrained', dest='pretrained', default=pretrain_flag, 
                        type=bool, metavar='PT_FLAG', help='use pre-trained model')
    # FLAG
    parser.add_argument('--ft', '--finetune', dest='finetune', default=finetune_flag,
                        type=bool, metavar='FT_FLAG', 
                        help='finetune the model, location specified by [--resume PATH]')
    # FLAG
    parser.add_argument('-e', '--evaluate', dest='evaluate', default=eval_flag,
                        type=bool, metavar='EVAL_FLAG', help='evaluate model on test set')
    # parser.add_argument('--half', dest='half', action='store_true',
    #                     help='use half-precision(16-bit) ')
    parser.add_argument('--save-dir', dest='save_dir',
                        help='The directory used to save the trained models',
                        default=os.path.join('model_checkpoints','save_temp'), type=str)
    ### ***IGNORE FOR NOW, COME BACK TO*** ###
    # parser.add_argument('--save-every', dest='save_every',
    #                     help='Saves checkpoints at every specified number of epochs',
    #                     type=int, default=10)
    # FLAG
    parser.add_argument('--sam', '--sam_segmentation', dest='use_sam', default=sam_flag,
                        type=bool, metavar='SAM_FLAG', help='use SAM for image segmentation')
    parser.add_argument('--seg_model', dest='seg_checkpoint', default=seg_path, 
                        type=str, metavar='PATH', help='path to segmentation model (default: none)')
    parser.add_argument('--mp', '--mask_padding_param', dest="mpp", default=mask_pad_param, type=int, 
                        metavar='N', help='padding width to extend mask border during segmentation')

    
    # Trick .ipynb notebook into properly compiling parse_args with empty "args" parameter
    config = parser.parse_args(args=[])
    #config = parser.parse_args()  

    return config

### Class Number to Name Dictionary

In [11]:
class_num_to_name_dict = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}

### AverageMeter Object Class

In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### Accuracy (precision@k) Function

In [13]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [14]:
def print_class_f1_scores(y_target, t_pred, end_flag=False):
    prec, rcll, f1sc, _ = pr_fscore_mtrc(y_target,t_pred,labels=np.arange(10,dtype=int),
                                         average=None,zero_division=0)
    print()
    if end_flag:
        print("Final Class F1-Scores:")
    else:
        print("Intermediate Class F1-Scores:")
    for c in np.arange(10,dtype=int):
        print(class_num_to_name_dict[c] + ": " + str(round(f1sc[c],4)))
    print()

### Save model checkpoint

In [15]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

### Train Function

In [16]:
def train(config, train_loader, model, criterion, optimizer, epoch, use_cuda):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            input = input.cuda()
            target = target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % config['print_freq'] == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))

### Evaluation Function

In [17]:
def evaluate(config, test_loader, model, criterion, use_cuda, seg_tf=None, norm_tf=None):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    running_targets = []
    running_preds = []

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            # Add current batch targets to running list of targets for computing class F1-Scores
            running_targets.extend(target.tolist())
            
            # Apply transformations during eval loop; segmentation, then normalization
            if seg_tf:
                input = torch.stack([seg_tf(input[i,:,:,:]) for i in range(input.shape[0])])
            if norm_tf:
                input = norm_tf(input)
                
            if use_cuda:
                input = input.cuda()
                target = target.cuda()

            # compute output
            output = model(input.float())
            loss = criterion(output, target)
            
            # Add current batch predictions to running list of predictions for computing class F1-Scores
            _, pred = output.topk(1, 1, True, True)
            running_preds.extend(pred.squeeze().tolist())
            
            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % config['print_freq'] == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))
                
            if i % (config['print_freq']*10) == 0:
                print_class_f1_scores(running_targets,running_preds)

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))
    print_class_f1_scores(running_targets,running_preds,end_flag=True)

    return top1.avg

### SAM Model and Transformer Class

In [22]:
import numpy as np
from rembg import remove
from PIL import Image, ImageDraw, ImageFilter, ImageChops
import os
import io

In [18]:
# # ViT_B SAM Model
# sam_b = sam_model_registry["vit_b"](checkpoint="eli_dev/seg_any_model/models/vit_b/sam_vit_b_01ec64.pth")
# mask_b_predictor = SamPredictor(sam_b)

In [19]:
# class SAMSegmentationTransform(object):
#     def __init__(self, mask_predictor, mask_padding=0):
#         self.predictor = mask_predictor
#         self.mask_padding = mask_padding
#         # If desired to extend object masks with padding
#         self.mask_pad_conv2d = None
#         if mask_padding > 0:
#             self.mask_pad_conv2d = nn.Conv2d(1, 1, kernel_size=(1+(2*mask_padding)), 
#                                              padding="same", bias=False)
#             self.mask_pad_conv2d.weight.data = torch.ones(1,1,(1+(2*mask_padding)),(1+(2*mask_padding)))
        
        
#     def __call__(self, image):
#         # Set image
#         self.predictor.set_image(image)
#         input_point = torch.Tensor([[16, 16]])
#         input_label = torch.Tensor([1])
#         masks, scores, logits = self.predictor.predict(
#             point_coords=input_point,
#             point_labels=input_label,
#             multimask_output=True,
#         )
        
#         # Identify best mask, extend borders if necessary, expand dims
#         best_mask = masks[torch.argmax(scores),:,:]
#         if self.mask_padding > 0:
#             best_mask = self.mask_pad_conv2d(best_mask)
#             best_mask[best_mask > 0] = 1
#         best_mask = torch.stack((best_mask,)*3, axis=-1)
        
#         seg_img = image * best_mask
#         seg_img[seg_img==0] = 255
#         seg_img = seg_img.int()
        
#         return seg_img

In [20]:
from pprint import pprint
import matplotlib.pyplot as plt

### "Main" Function, aka where most of the sequential logic and controlled

In [34]:
def main(inputFolder,outputFolder):
    #adding center focus that won't be removed
    def createEdgeMask(image, borderThickness=50):
        width, height = image.size
        mask = Image.new("L", (width, height), 0)
        draw = ImageDraw.Draw(mask)
        draw.rectangle(
            [borderThickness, borderThickness, width - borderThickness, height - borderThickness],
            fill=255
        )
        mask = ImageChops.invert(mask)
        return mask
    #
    def imageMask(inputPath, outputPath, borderThickness=50):
        with open(inputPath, 'rb') as inputFile:
            inputData = inputFile.read()
            outputData = remove(inputData)
        originalImage = Image.open(inputPath).convert("RGBA")
        processedImage = Image.open(io.BytesIO(outputData)).convert("RGBA")
        edgeMask = createEdgeMask(originalImage, borderThickness=borderThickness)
        result = Image.composite(processedImage, originalImage, edgeMask)
        processedImage.save(outputPath)

    print("Masking...")
    for fileName in os.listdir(inputFolder):
        if fileName.lower().endswith(('png', 'jpg', 'jpeg')):
            inputPath = os.path.join(inputFolder, fileName)
            outputPath = os.path.join(outputFolder, fileName)
            print("Processing: " +str(fileName))
            imageMask(inputPath, outputPath, borderThickness=50)


In [35]:
main(inputFolder="dataset",outputFolder="out")

Processing patched_3.png
Processing patched_2.png
Processing patched_0.png
Processing patched_1.png
Processing patched_5.png
Processing patched_4.png
Processing patched_6.png
Processing patched_7.png
Processing patched_11.png
Processing patched_10.png
Processing patched_12.png
Processing patched_13.png
Processing patched_14.png
Processing patched_15.png
Processing patched_9.png
Processing patched_8.png


In [ ]:
# eli_dev/seg_any_model/models/vit_h/sam_vit_h_4b8939.pth # 8 images: about 3 min, 36 sec
# eli_dev/seg_any_model/models/vit_l/sam_vit_l_0b3195.pth # 8 images: about 2 min, 41 sec
# eli_dev/seg_any_model/models/vit_b/sam_vit_b_01ec64.pth # 8 images: about 1 min, 16 sec
main(pt_path="model_checkpoints/pretrained/resnet56-4bfd9763.th",pretrain_flag=True,
     finetune_flag=False,eval_flag=True,sam_flag=True,batch_size=8,print_freq=1,
     seg_path="eli_dev/seg_any_model/models/vit_l/sam_vit_l_0b3195.pth",mask_pad_param=1)

CUDA Environment Unavailable; Running on CPU
=> loading checkpoint 'model_checkpoints/pretrained/resnet56-4bfd9763.th'
Files already downloaded and verified
